# Setup

## Check GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-19726555-1d6a-3e5d-15c7-0793a335e6cb)


## Config

In [2]:
RANDOM_SEED = 7

GDRIVE_DIR = "/content/gdrive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/My Drive"
GDRIVE_PROJECT_DIR = GDRIVE_HOME_DIR + "/causal-mechanisms-on-pointclouds"
GDRIVE_CKPTS_PATH = GDRIVE_PROJECT_DIR + "/checkpoints"
GDRIVE_DATASET = GDRIVE_PROJECT_DIR + "/data/mnist"

## Link to GDrive

In [3]:
# link google drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

%cd $GDRIVE_PROJECT_DIR

Mounted at /content/gdrive
/content/gdrive/My Drive/causal-mechanisms-on-pointclouds


In [4]:
!pip install wandb
!wandb login

     |████████████████████████████████| 1.8MB 8.4MB/s 
     |████████████████████████████████| 174kB 50.8MB/s 
     |████████████████████████████████| 143kB 43.5MB/s 
     |████████████████████████████████| 102kB 14.4MB/s 
     |████████████████████████████████| 143kB 48.3MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=cb18b9e4b85ef1abc686ed3afb7bc7351a4abb7128c8ac786a7cb915c1cf92f1
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=16b1a0c8e66bb7d8a7baf40f1fc7f8cd5550656e996f210f6b35d2a9402136a4
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll h

## Install dependencies

In [5]:
!pip install ipython-autotime
%load_ext autotime

time: 164 µs (started: 2021-07-16 13:16:49 +00:00)


In [6]:
!pip install Ninja

     |████████████████████████████████| 112kB 8.1MB/s 
time: 2.83 s (started: 2021-07-16 13:16:49 +00:00)


In [7]:
# !pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 torchaudio===0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
import torch
TORCH_VERSION = torch.__version__[:5]
if TORCH_VERSION[:3] == '1.8':
  TORCH_VERSION = '1.8.0'
CUDA_VERSION = torch.version.cuda.replace('.','')
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-{TORCH_VERSION}+cu{CUDA_VERSION}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-{TORCH_VERSION}+cu{CUDA_VERSION}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH_VERSION}+cu{CUDA_VERSION}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH_VERSION}+cu{CUDA_VERSION}.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 2.6MB 6.7MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 1.4MB 7.4MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 931kB 6.2MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 389kB 5.0MB/s 
     |████████████████████████████████| 225kB 8.0MB/s 
     |████████████████████████████████| 235kB 12.9MB/s 
     |████████████████████████████████| 133kB 13.7MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.7.2-cp37-none-any.whl size=388143 sha256=e2e5781abd175122b27b74f56af338792803168d7d3387f14e7339475a24cfd0
  Stored in directory: /root/.cache/pip/wheels/e7/66/5b/ad17ef7f04b7c425dc6930daac160c374

## Import

In [8]:
import numpy as np
import random
from tqdm.autonotebook import tqdm

import wandb

from torch_geometric.data import DataLoader, Batch
import torch_geometric.transforms as T

from src.arguments import Args
from src.mechanisms import GaussianNoise
# from data.modelnet import TransformedModelNet
from src.datasets.mnist import MNISTPointCloud
from src.trainer import Trainer
from src.model import Expert, Discriminator
from src.utils import plot_mesh, plot_pointcloud, tranformations_distribution

time: 9.38 s (started: 2021-07-16 13:17:13 +00:00)


## Seed

In [9]:
import os
torch.set_default_tensor_type(torch.FloatTensor)

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(RANDOM_SEED)

# torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

time: 10.1 ms (started: 2021-07-16 13:17:22 +00:00)


# Dataset

In [10]:
# store this id to use it later when resuming
# id_wandb = wandb.util.generate_id()
# print(id_wandb)
id_wandb = "1vjyupir"

time: 2.46 ms (started: 2021-07-16 13:17:22 +00:00)


In [11]:
# transforms=[T.RandomScale([0.5, 0.5]), T.RandomShear(0.5), GaussianNoise(0, 1)]
transforms=[T.RandomShear(0.5)]

args = Args(id_wandb=id_wandb, batch_size=20, num_experts=len(transforms), initialize=False, epochs_init=5, \
            dataset_path=GDRIVE_DATASET, project_dir=GDRIVE_PROJECT_DIR, models_path=GDRIVE_CKPTS_PATH)

time: 1.7 ms (started: 2021-07-16 13:17:22 +00:00)


In [12]:
pre_transform = T.SamplePoints(args.num_points, remove_faces=False)

train_dataset = MNISTPointCloud(GDRIVE_DATASET, transforms=transforms, pre_transform=pre_transform)
test_dataset = MNISTPointCloud(GDRIVE_DATASET, transforms=transforms, train=False, pre_transform=pre_transform)

time: 51.2 s (started: 2021-07-16 13:17:22 +00:00)


In [13]:
train_dataset.idx_to_transf

{0: RandomShear(0.5)}

time: 4.71 ms (started: 2021-07-16 13:18:14 +00:00)


In [14]:
print(train_dataset[0])
print(train_dataset[1])

TransformedSample(face=[3, 232], face_transf=[3, 221], original_pos=[166, 3], original_pos_transf=[155, 3], pos=[1024, 3], pos_transf=[1024, 3], transf=[1], y=[1], y_transf=[1])
TransformedSample(face=[3, 241], face_transf=[3, 300], original_pos=[176, 3], original_pos_transf=[203, 3], pos=[1024, 3], pos_transf=[1024, 3], transf=[1], y=[1], y_transf=[1])
time: 415 ms (started: 2021-07-16 13:18:14 +00:00)


In [15]:
tranformations_distribution(train_dataset)
tranformations_distribution(test_dataset)


{RandomShear(0.5): 60000}



{RandomShear(0.5): 10000}
time: 15.6 s (started: 2021-07-16 13:18:14 +00:00)


## Visualization

In [16]:
train_dataset.idx_to_transf

{0: RandomShear(0.5)}

time: 4.62 ms (started: 2021-07-16 13:18:30 +00:00)


In [17]:
print(train_dataset[0].transf)
plot_pointcloud(train_dataset[0].pos_transf)
plot_pointcloud(train_dataset[0].pos)
plot_mesh(train_dataset[0].original_pos, train_dataset[0].face)

tensor([0])


time: 1.26 s (started: 2021-07-16 13:18:30 +00:00)


In [18]:
print(train_dataset[1].transf)
plot_pointcloud(train_dataset[1].pos_transf)
plot_pointcloud(train_dataset[1].pos)
plot_mesh(train_dataset[1].original_pos, train_dataset[1].face)

tensor([0])


time: 517 ms (started: 2021-07-16 13:18:31 +00:00)


# Model

In [19]:
discriminator = Discriminator(args)
expert = Expert(args)

time: 71.8 ms (started: 2021-07-16 13:18:31 +00:00)


# Train

## Initialization Trainer

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

time: 1.45 ms (started: 2021-07-16 13:18:32 +00:00)


In [21]:
trainer = Trainer(hparams=args, expert=expert, discriminator=discriminator, train_dataset=train_dataset, test_dataset=test_dataset)

print(count_parameters(trainer.discriminator))
print(count_parameters(trainer.expert))

2536065
884035
time: 39.9 s (started: 2021-07-16 13:18:32 +00:00)


In [22]:
if args.initialize:
  trainer.initialize_expert()
else:
  ckpt = torch.load(f'{args.models_path}/init/expert.pth', map_location=args.device)
  trainer.expert.load_state_dict(ckpt['model'])
  # trainer.E_optimizer.load_state_dict(ckpt['optim'])

time: 808 ms (started: 2021-07-16 13:19:11 +00:00)


#### Identity visualization

In [23]:
sample = train_dataset[0]
batch = Batch.from_data_list([sample], follow_batch=['pos', 'pos_transf']).to(args.device)
batch

with torch.no_grad():
  expert = trainer.expert.to(args.device)
  expert.eval()

  new_pos = expert(batch.pos_transf, batch.pos_transf_batch).to("cpu")
  plot_pointcloud(sample.pos_transf)
  plot_pointcloud(new_pos)

time: 1.2 s (started: 2021-07-16 13:19:12 +00:00)


## Training

In [ ]:
trainer.load_checkpoint()
with torch.autograd.set_detect_anomaly(True):
 trainer.train()

Loading checkpoint...


wandb: Currently logged in as: santonelli7 (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning:

Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)



In [ ]:
 # from src.utils import plot_trend_scores
 # 
 # trainer.load_checkpoint(5)
 # plot_trend_scores(torch.stack(trainer.trend_scores), trainer.train_dataset.idx_to_transf)

# Test

In [ ]:
# !pip install plotly==4.7.1
# !wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
# !chmod +x /usr/local/bin/orca
# !apt-get install xvfb libgtk2.0-0 libgconf-2-4
# import plotly.graph_objects as go

In [ ]:
# trainer.test()